In [ ]:
from prodock.io.pdb_query import PDBQuery
proc = PDBQuery("5N2F", "Data/testcase", chains=["A", "B"], ligand_code="8HW")
proc.run_all()
print("raw:", proc.cocrystal_ligand_path)
print("processed:", proc.reference_ligand_path)

In [ ]:
from prodock.preprocess.gridbox import GridBox
from prodock.vis.provis import ProVis
from prodock.vis.provis_gui import ProVisGUI


In [ ]:
gb = GridBox().load_ligand("Data/testcase/reference_ligand/8HW.sdf", fmt="sdf") \
              .from_ligand_pad(pad=4.0, isotropic=True)

print("Box dict for Vina:", gb.vina_dict)
print("Box config snippet:\n", gb.to_vina_lines())


In [ ]:
viz = ProVis(800, 600)
viz.load_receptor("Data/testcase/filtered_protein/5N2F.pdb") \
   .set_receptor_style("cartoon", "white") \
   .load_ligand("Data/testcase/reference_ligand/8HW.sdf", fmt="sdf") \
   .highlight_ligand(style="stick", color="cyan") \
   .add_gridbox_from(gb, labels=True) \
   .show()


In [ ]:
gui = ProVisGUI().build().display()


In [2]:
"""
protein_process.py
------------------

ProteinProcess: fixes PDBs and performs two-stage energy minimization (gas + optional solvent).

Features:
- Fix missing atoms/hydrogens with PDBFixer.
- Gas-phase minimization with backbone restraints (N, CA, C).
- Optional explicit-water minimization (TIP3P) with ionic strength.
- Optional conversion to PDBQT using mekoo wrapper `mk_prepare_receptor.py`.
- PyMOL postprocessing (renumber/residue offset, remove solvent/ions) if PyMOL available.

Usage (example):
    from protein_process import ProteinProcess
    pp = ProteinProcess()
    pp.fix_and_minimize_pdb(
        input_pdb="input.pdb",
        output_dir="out",
        energy_diff=10.0,
        max_minimization_steps=5000,
        start_at=1,
        ion_conc=0.15,
        cofactors=["HEM"],
        pdb_id="5N2F",
        protein_name="5N2F_fixed",
        minimize_in_water=True,
        out_fmt="pdbqt",
    )
    print(pp.final_pdb_path)
"""

from __future__ import annotations

import os
import shutil
import logging
import subprocess
from pathlib import Path
from typing import Optional, Dict, Any, List, Tuple

# PyMOL import is optional (used only for postprocessing). If missing, we continue without it.
try:
    from pymol import cmd  # type: ignore
except Exception:  # pragma: no cover - optional runtime dep
    cmd = None  # type: ignore

# PDBFixer and OpenMM imports
from pdbfixer import PDBFixer
from openmm.app import PDBFile, Modeller, ForceField, Simulation, PME, NoCutoff, HBonds
from openmm import Platform, CustomExternalForce
from openmm import LangevinIntegrator
from openmm.unit import nanometer, kilojoule_per_mole, kelvin, picosecond, molar

logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())


class ProteinProcess:
    """
    Helper class for fixing and minimizing protein PDBs.

    The main entry is :py:meth:`fix_and_minimize_pdb` which performs:
      1. PDB fixing using PDBFixer (missing residues/atoms/hydrogens)
      2. Gas-phase minimization with backbone restraints
      3. (Optional) Solvent minimization using TIP3P water + ions
      4. Optional conversion to PDBQT via external mekoo wrapper
      5. PyMOL postprocessing (renumber residues, remove solvent except cofactors, remove NaCl)

    :ivar final_pdb_path: Path to the final saved PDB/PDBQT (set after running).
    :ivar last_simulation_report: Dict with summary info about the last run.
    """

    def __init__(self) -> None:
        """
        Initialise ProteinProcess.
        """
        self.final_pdb_path: Optional[Path] = None
        self.last_simulation_report: Optional[Dict[str, Any]] = None

    @staticmethod
    def _choose_platform() -> Tuple[Platform, Dict[str, str]]:
        """
        Try CUDA -> OpenCL -> CPU platforms and return (platform, properties).

        :returns: (Platform instance, properties dict)
        """
        for name in ("CUDA", "OpenCL", "CPU"):
            try:
                plat = Platform.getPlatformByName(name)
                if name == "CUDA":
                    props = {"CudaPrecision": "mixed"}
                elif name == "OpenCL":
                    props = {}
                else:
                    props = {}
                logger.info("Using OpenMM platform: %s", name)
                return plat, props
            except Exception:
                continue
        # Fallback: return CPU Platform (guaranteed to exist)
        plat = Platform.getPlatformByName("CPU")
        return plat, {}

    def fix_and_minimize_pdb(
        self,
        input_pdb: str,
        output_dir: str,
        energy_diff: float = 10.0,
        max_minimization_steps: int = 5000,
        start_at: int = 1,
        ion_conc: float = 0.15,
        cofactors: Optional[List[str]] = None,
        pdb_id: Optional[str] = None,
        protein_name: Optional[str] = None,
        minimize_in_water: bool = False,
        backbone_k_kcal_per_A2: float = 5.0,
        out_fmt: str = "pdb",
        mekoo_cmd: Optional[str] = "mk_prepare_receptor.py",
    ) -> "ProteinProcess":
        """
        Fix PDB (missing atoms/hydrogens) and perform minimization.

        The method runs a gas-phase minimization with backbone restraints (N, CA, C)
        and optionally performs a second minimization after adding water + ions.

        Units & notes:
        - `energy_diff` is the minimizer tolerance in **kJ/mol** (OpenMM expects kJ/mol).
        - `ion_conc` is in molar (M). e.g. 0.15 -> 150 mM.
        - `backbone_k_kcal_per_A2` is restraint spring constant in kcal/(mol·Å²) (converted internally).

        :param input_pdb: Path to input PDB file.
        :param output_dir: Directory where outputs will be written (created if missing).
        :param energy_diff: Minimizer tolerance (kJ/mol).
        :param max_minimization_steps: Maximum iterations for the minimizer.
        :param start_at: Residue numbering start offset applied in final PyMOL step.
        :param ion_conc: Ionic strength in molar (M) for solvent minimization.
        :param cofactors: List of residue names to preserve as cofactors when cleaning solvent.
        :param pdb_id: Optional PDB identifier (used for naming).
        :param protein_name: Optional friendly name (preferred for output filename).
        :param minimize_in_water: If True, perform the second minimization in explicit water.
        :param backbone_k_kcal_per_A2: backbone restraint spring constant in kcal/(mol·Å²).
        :param out_fmt: 'pdb' (default) or 'pdbqt'. If 'pdbqt', tries to run mekoo wrapper to produce PDBQT.
        :param mekoo_cmd: The command/executable name for mk_prepare_receptor.py (default 'mk_prepare_receptor.py').
        :returns: self
        :raises RuntimeError: on unrecoverable errors (OpenMM or PDBFixer missing, etc).
        """
        output_dirpath = Path(output_dir)
        output_dirpath.mkdir(parents=True, exist_ok=True)

        base_name = protein_name or pdb_id or Path(input_pdb).stem or "protein"
        final_pdb = output_dirpath / f"{base_name}.pdb"
        tmp_gas_pdb = output_dirpath / f"{base_name}_gas_min_tmp.pdb"

        logger.info("Fixing and minimizing %s -> %s", input_pdb, final_pdb)

        # -------------------------
        # Step 1: Fix PDB with PDBFixer
        # -------------------------
        fixer = PDBFixer(filename=input_pdb)
        fixer.removeHeterogens(keepWater=False)
        fixer.findMissingResidues()
        fixer.findMissingAtoms()
        fixer.addMissingAtoms()
        fixer.addMissingHydrogens(pH=7.4)

        modeller = Modeller(fixer.topology, fixer.positions)

        # -------------------------
        # Step 2: Gas-phase minimization (NoCutoff, backbone restraints)
        # -------------------------
        logger.info("Starting gas-phase minimization (NoCutoff)")

        forcefield = ForceField("amber14-all.xml")
        system = forcefield.createSystem(modeller.topology, nonbondedMethod=NoCutoff, constraints=HBonds)

        # Convert backbone k from kcal/(mol·Å²) to kJ/(mol·nm²)
        # 1 kcal/mol = 4.184 kJ/mol ; 1 Å = 0.1 nm => per Å^2 -> multiply by 1/(0.1^2)=100
        k_kj_per_mol_nm2 = backbone_k_kcal_per_A2 * 4.184 * 100.0

        force = CustomExternalForce("0.5 * k * ((x-x0)^2 + (y-y0)^2 + (z-z0)^2)")
        force.addPerParticleParameter("k")
        force.addPerParticleParameter("x0")
        force.addPerParticleParameter("y0")
        force.addPerParticleParameter("z0")

        # Ensure positions are Quantity[nanometer]
        def _to_nm(val) -> float:
            if hasattr(val, "value_in_unit"):
                return val.value_in_unit(nanometer)
            # plain float → assume it's already in nm
            return float(val)
        for atom in modeller.topology.atoms():
            if atom.name in ("N", "CA", "C"):
                idx = atom.index
                pos = modeller.positions[idx]
                x0 = pos.x.value_in_unit(nanometer)
                y0 = pos.y.value_in_unit(nanometer)
                z0 = pos.z.value_in_unit(nanometer)


                x0 = _to_nm(pos.x)
                y0 = _to_nm(pos.y)
                z0 = _to_nm(pos.z)
                force.addParticle(idx, [k_kj_per_mol_nm2, x0, y0, z0])

        system.addForce(force)

        platform, plat_props = self._choose_platform()
        integrator = LangevinIntegrator(300 * kelvin, 1.0 / picosecond, 0.002 * picosecond)
        integrator.setRandomNumberSeed(42)

        simulation = Simulation(modeller.topology, system, integrator, platform, plat_props)
        simulation.context.setPositions(modeller.positions)

        try:
            logger.info("Minimizing (gas): tolerance=%s kJ/mol, maxIter=%s", energy_diff, max_minimization_steps)
            simulation.minimizeEnergy(tolerance=energy_diff, maxIterations=max_minimization_steps)
        except Exception as exc:
            logger.exception("Gas-phase minimization failed: %s", exc)
            raise RuntimeError("Gas-phase minimization failed") from exc

        state = simulation.context.getState(getPositions=True)
        minimized_positions = state.getPositions()

        with open(tmp_gas_pdb, "w") as f:
            PDBFile.writeFile(simulation.topology, minimized_positions, f)
        logger.info("Saved gas-minimized PDB to %s", tmp_gas_pdb)

        report: Dict[str, Any] = {
            "stage": "gas",
            "platform": simulation.context.getPlatform().getName(),
            "tolerance_kJ_per_mol": energy_diff,
            "max_minimization_steps": max_minimization_steps,
        }

        # -------------------------
        # Step 3: Optional solvent minimization
        # -------------------------
        if minimize_in_water:
            logger.info("Starting solvent minimization (adding TIP3P water, ions=%s M)", ion_conc)
            gas_pdb = PDBFile(str(tmp_gas_pdb))
            modeller = Modeller(gas_pdb.topology, gas_pdb.positions)

            ff_with_water = ForceField("amber14-all.xml", "amber14/tip3p.xml")
            modeller.addSolvent(ff_with_water, model="tip3p", padding=1.0 * nanometer,
                                ionicStrength=ion_conc * molar)

            system = ff_with_water.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)

            integrator = LangevinIntegrator(300 * kelvin, 1.0 / picosecond, 0.002 * picosecond)
            integrator.setRandomNumberSeed(42)

            simulation = Simulation(modeller.topology, system, integrator, platform, plat_props)
            simulation.context.setPositions(modeller.positions)

            try:
                logger.info("Minimizing (solvent): tolerance=%s kJ/mol, maxIter=%s", energy_diff, max_minimization_steps)
                simulation.minimizeEnergy(tolerance=energy_diff, maxIterations=max_minimization_steps)
            except Exception as exc:
                logger.exception("Solvent minimization failed: %s", exc)
                raise RuntimeError("Solvent minimization failed") from exc

            state = simulation.context.getState(getPositions=True)
            minimized_positions = state.getPositions()

            with open(final_pdb, "w") as f:
                PDBFile.writeFile(simulation.topology, minimized_positions, f)
            logger.info("Saved solvent-minimized PDB to %s", final_pdb)
            report["stage"] = "solvent"
        else:
            shutil.copy2(tmp_gas_pdb, final_pdb)
            logger.info("Copied gas-minimized PDB to final location %s", final_pdb)

        try:
            if tmp_gas_pdb.exists():
                tmp_gas_pdb.unlink()
        except Exception:
            logger.debug("Could not remove temporary gas pdb: %s", tmp_gas_pdb)

        # -------------------------
        # Step 4: Optional PDBQT conversion via mekoo wrapper
        # -------------------------
        final_artifact = final_pdb
        if out_fmt and out_fmt.lower() == "pdbqt":
            pdbqt_path = final_pdb.with_suffix(".pdbqt")
            mk_cmd = shutil.which(mekoo_cmd) or mekoo_cmd  # allow absolute path or rely on PATH
            mk_args = [mk_cmd, "--pdbqt", str(pdbqt_path), "-o", str(pdbqt_path), "--skip_gpf"]
            try:
                logger.info("Attempting PDBQT preparation via: %s", " ".join(mk_args))
                # suppress stdout/stderr to keep notebook clean
                with open(os.devnull, "wb") as devnull:
                    completed = subprocess.run(mk_args, stdout=devnull, stderr=devnull, check=False)
                if completed.returncode == 0 and pdbqt_path.exists():
                    final_artifact = pdbqt_path
                    logger.info("PDBQT created: %s", pdbqt_path)
                else:
                    logger.warning(
                        "mk_prepare_receptor.py did not create PDBQT (returncode=%s). Keeping PDB: %s",
                        completed.returncode,
                        final_pdb,
                    )
            except FileNotFoundError:
                logger.warning("mekoo wrapper not found (mk_prepare_receptor.py). Skipping pdbqt step.")
            except Exception as exc:
                logger.warning("PDBQT conversion failed: %s. Keeping PDB.", exc)

        # -------------------------
        # Step 5: PyMOL cleanup: renumber residues & remove solvent (leave cofactors)
        # -------------------------
        if cmd is not None and final_artifact.suffix.lower() == ".pdb":
            try:
                logger.info("Running PyMOL postprocessing (renumber, remove solvent/ions)")
                cmd.load(str(final_artifact))
                offset = int(start_at) - 1
                cmd.alter("all", f"resi=str(int(resi)+{offset})")
                if cofactors:
                    cof_sel = " or ".join([f"resn {c}" for c in cofactors])
                    cmd.select("cofactors", cof_sel)
                    cmd.select("removed_solvent", "solvent and not cofactors")
                else:
                    cmd.select("removed_solvent", "solvent")
                cmd.remove("removed_solvent")
                cmd.select("nacl", "resn NA or resn CL")
                cmd.remove("nacl")
                cmd.save(str(final_artifact), "all")
                cmd.delete("all")
                logger.info("PyMOL postprocessing complete, saved %s", final_artifact)
            except Exception as exc:
                logger.warning("PyMOL postprocessing failed (is PyMOL available?): %s", exc)
        else:
            if cmd is None:
                logger.info("PyMOL not available; skipping postprocessing.")
            else:
                logger.info("Final artifact is not a PDB; skipping PyMOL postprocessing.")

        # finalize object state and return
        self.final_pdb_path = final_artifact
        report.update(
            {
                "final_artifact": str(final_artifact),
                "minimized_in_water": bool(minimize_in_water),
                "out_fmt": out_fmt,
            }
        )
        self.last_simulation_report = report

        logger.info("Finished fix_and_minimize_pdb -> %s", final_artifact)
        return self

    def help(self) -> None:
        """
        Print docstring / usage help.
        """
        print(self.__class__.__doc__)

    def __repr__(self) -> str:
        return f"<ProteinProcess final={self.final_pdb_path}>"

# # CLI
# if __name__ == "__main__":  # pragma: no cover - convenience CLI
#     import argparse

#     parser = argparse.ArgumentParser(description="Fix and minimize a PDB with ProteinProcess")
#     parser.add_argument("input_pdb", help="Input PDB file")
#     parser.add_argument("--out", "-o", default="out", help="Output directory")
#     parser.add_argument("--energy_diff", type=float, default=10.0, help="Minimizer tolerance (kJ/mol)")
#     parser.add_argument("--max_steps", type=int, default=5000, help="Maximum minimizer iterations")
#     parser.add_argument("--start_at", type=int, default=1, help="Residue numbering start")
#     parser.add_argument("--ion_conc", type=float, default=0.15, help="Ionic strength (M)")
#     parser.add_argument("--cofactors", type=str, default="", help="Comma-separated cofactors to preserve (e.g. HEM)")
#     parser.add_argument("--pdb_id", type=str, default=None, help="PDB id / basename (optional)")
#     parser.add_argument("--prot_name", type=str, default=None, help="Protein name for output (optional)")
#     parser.add_argument("--water", action="store_true", help="Perform minimization in water (explicit solvent)")
#     parser.add_argument("--out-fmt", choices=["pdb", "pdbqt"], default="pdb", help="Output format (pdb or pdbqt)")
#     parser.add_argument("--mekoo-cmd", type=str, default="mk_prepare_receptor.py", help="mekoo wrapper command name/path")
#     args = parser.parse_args()

#     cofactors = [c.strip() for c in args.cofactors.split(",")] if args.cofactors else []
#     pp = ProteinProcess()
#     pp.fix_and_minimize_pdb(
#         input_pdb=args.input_pdb,
#         output_dir=args.out,
#         energy_diff=args.energy_diff,
#         max_minimization_steps=args.max_steps,
#         start_at=args.start_at,
#         ion_conc=args.ion_conc,
#         cofactors=cofactors,
#         pdb_id=args.pdb_id,
#         protein_name=args.prot_name,
#         minimize_in_water=args.water,
#         out_fmt=args.out_fmt,
#         mekoo_cmd=args.mekoo_cmd,
#     )
#     print("Final artifact:", pp.final_pdb_path)


In [3]:
# from protein_process import ProteinProcess

pp = ProteinProcess()

pp.fix_and_minimize_pdb(
    input_pdb="Data/testcase/filtered_protein/5N2F.pdb",     # input PDB file
    output_dir="Data/testcase/filtered_protein/",     # where to save minimized structure
    energy_diff=10.0,         # minimizer tolerance (kJ/mol)
    max_minimization_steps=5000,
    pdb_id="5N2F",            # optional PDB id
    protein_name="5N2F", # friendly output name
    minimize_in_water=False,  # only gas-phase minimization
)

print("Final minimized file:", pp.final_pdb_path)
print("Summary:", pp.last_simulation_report)


AttributeError: 'float' object has no attribute 'value_in_unit'